In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd

RDATA = os.path.join('data', 'raw')
PDATA = os.path.join('data', 'processed')

#### The notebook prepares two analysis datasets from public data sources
Input data sources:
- [Iowa ABD liquor sales records](https://bit.io/bitdotio/iowa_liquor_sales)
- [Iowa state level unemployment from BLS and county-level unemployment from LAUS](https://bit.io/bitdotio/iowa_unemployment)
- [2019 5-Year American Community Survey from US Census Bureau](https://bit.io/bitdotio/iowa_county_2019_5Y_acs)
- [New York Times COVID data compilation](https://bit.io/bitdotio/nytimes_covid)

Output datasets:
- Monthly state-level series of liquor sales, unemployment, and COVID stats joined to static census attributes
- Monthly county-level series of liquor sales, unemployment, and COVID stats joined to static census attributes

#### State-level series at monthly frequency

In [2]:
# Begin by preparing the liquor sales data at state level with monthly frequency
df_sales = pd.read_csv(os.path.join(RDATA, 'iowa_liquor_sales.csv'), low_memory=False, parse_dates=['Date'])

# Drop any duplicated transactions
df_sales = df_sales.drop_duplicates()

# Cast imestamps to monthly periods
df_sales['year_month'] = df_sales['Date'].dt.to_period('M')

# There are some erroneous entries breaking numeric aggs, remove those rare records
agg_cols = ['Volume Sold (Liters)', 'Bottles Sold']
for col in agg_cols:
    df_sales[col] = pd.to_numeric(df_sales[col], errors='coerce')

# Check for reasonable data loss
before = df_sales.shape[0]
df_sales = df_sales.dropna(subset=agg_cols)
after = df_sales.shape[0]
print(f'Dropped {before - after} of {before} ({(before - after) / before:.3f}%) records due to missing data.')

# Agg state-level dataset
agg_map = {
    'Volume Sold (Liters)': np.sum,
    'Invoice/Item Number': len,
    'Bottles Sold': np.sum    
}
rename_map = {
    'Volume Sold (Liters)': 'n_liters',
    'Invoice/Item Number': 'n_transactions',
    'Bottles Sold': 'n_bottles'
}
df_state = df_sales.groupby('year_month')\
                   .agg(agg_map)\
                   .rename(columns=rename_map)

# Feature construction
df_state['bottles_p_transaction'] = df_state['n_bottles'] / df_state['n_transactions']

df_state.head()

Dropped 0 of 21190664 (0.000%) records due to missing data.


,n_liters,n_transactions,n_bottles,bottles_p_transaction
year_month,,,,
2012-01,1212082.49,147770,1302231,8.812553
2012-02,1386565.53,156728,1457824,9.301618
2012-03,1352873.00,160535,1458665,9.086274
2012-04,1499566.78,164970,1588914,9.631533
2012-05,1709432.64,192270,1782175,9.269127


In [3]:
# Next, add in state-level population and (seasonally adjusted) employment statistics
df_ia_emp = pd.read_csv(os.path.join(RDATA, 'statewide_seasonally_adjusted.csv'))
df_ia_emp['year_month'] = (df_ia_emp['Year'].astype(str) + '-' + df_ia_emp['Period'])\
                                            .apply(lambda x: pd.Period(x, 'M'))
df_ia_emp = df_ia_emp.set_index('year_month')
for col in ['unemployment rate', 'employment', 'employment-population ratio', 'unemployment']:
    df_ia_emp[col] = df_ia_emp[col].str.replace(r'\(.*\)', '', regex=True)\
                                   .str.strip()\
                                   .astype(float)
    
# I might want to compare these population figures with what I get from the 2019 ACS
df_ia_emp['population'] = 100 * df_ia_emp['employment'] / df_ia_emp['employment-population ratio']

# It's worth noting that this population is much lower than total IA pop, likely due to only including 
# A segment consider in the eligible-for-labor segment

# Rename columns
df_ia_emp = df_ia_emp.rename(columns={
    'unemployment rate': 'unemployment_rate',
    'population': 'emp_population'})

In [4]:
# Next, get COVID rates for entire state

# Get latest cumulative cases and deaths per month, per county 
# Agg per month at state level

df_cov = pd.read_csv(os.path.join(RDATA, 'us_counties.csv'), parse_dates=['date'])
df_cov = df_cov.sort_values('date')
df_cov['year_month'] = df_cov['date'].dt.to_period('M')
for col in ['cases', 'deaths']:
    df_cov[f'cum_{col}'] = df_cov.groupby(['year_month', 'county', 'state'])[col].transform('last')
df_cov.head()

# Get IA total by month
df_ia_cov = df_cov.loc[df_cov['state'] == 'Iowa'][['year_month', 'cum_cases', 'cum_deaths']]\
                  .drop_duplicates()\
                  .groupby('year_month')\
                  .agg({'cum_cases': np.sum, 
                        'cum_deaths': np.sum})

# Compute diffs and fillna with initial value
for col in ['cases', 'deaths']:
    df_ia_cov[col] = df_ia_cov[f'cum_{col}'].diff()
    df_ia_cov[col] = df_ia_cov[col].fillna(df_ia_cov[f'cum_{col}'])
df_ia_cov.tail()

,cum_cases,cum_deaths,cases,deaths
year_month,,,,
2021-01,317755,4877.0,36043.0,986.0
2021-02,336455,5470.0,18700.0,593.0
2021-03,351402,5743.0,14947.0,273.0
2021-04,364841,5950.0,13439.0,207.0
2021-05,370625,6035.0,5784.0,85.0


In [5]:
# Join all features together and store State series dataset
keep_cols = [
    'n_liters',
    'n_transactions',
    'n_bottles',
    'bottles_p_transaction',
    'employment',
    'unemployment',
    'unemployment_rate',
    'emp_population',
    'cases',
    'deaths'
]

# Add in state population >18yo from 2019 ACS
pop_totals = {
    'pop_total': 3155070,
    'pop_gt18': 2433240,
    'pop_gt21': 2294466
}

df_state_series = df_state.join(df_ia_emp, how='left').join(df_ia_cov, how='left')[keep_cols]
df_state_series['time_index'] = np.arange(df_state_series.shape[0])
df_state_series['time_cycle'] = np.sin(2 * np.pi * (np.array(df_state_series.index.to_timestamp().month) - 1) / 11)
df_state_series = df_state_series.reset_index(drop=False)
for key, value in pop_totals.items():
    df_state_series[key] = value
df_state_series.to_csv(os.path.join(PDATA, 'df_state_series.csv'), index=False)
df_state_series.tail()

,year_month,n_liters,n_transactions,n_bottles,bottles_p_transaction,employment,unemployment,unemployment_rate,emp_population,cases,deaths,time_index,time_cycle,pop_total,pop_gt18,pop_gt21
107,2020-12,2.360986e+06,253889,2912674,11.472234,1559562.0,59879.0,3.7,2.483379e+06,51078.0,1465.0,107,-1.133108e-15,3155070,2433240,2294466
108,2021-01,1.724365e+06,196467,2203381,11.215018,1567143.0,58010.0,3.6,2.483586e+06,36043.0,986.0,108,0.000000e+00,3155070,2433240,2294466
109,2021-02,1.773047e+06,190006,2225257,11.711509,1570232.0,60695.0,3.7,2.480619e+06,18700.0,593.0,109,5.406408e-01,3155070,2433240,2294466
110,2021-03,2.125044e+06,229986,2680901,11.656801,1573432.0,60875.0,3.7,2.481754e+06,14947.0,273.0,110,9.096320e-01,3155070,2433240,2294466
111,2021-04,2.034895e+06,224667,2643103,11.764536,NaN,NaN,NaN,NaN,13439.0,207.0,111,9.898214e-01,3155070,2433240,2294466


#### County-level series at monthly frequency

In [6]:
# Begin by preparing the liquor sales data at county level with monthly frequency
df_sales = pd.read_csv(os.path.join(RDATA, 'iowa_liquor_sales.csv'), low_memory=False, parse_dates=['Date'])
df_sales = df_sales.drop_duplicates()
df_sales['year_month'] = df_sales['Date'].dt.to_period('M')

# There are some erroneous entries breaking numeric aggs and missing counties, remove those rare records
agg_cols = ['Volume Sold (Liters)', 'Bottles Sold', 'County Number']
for col in agg_cols:
    df_sales[col] = pd.to_numeric(df_sales[col], errors='coerce')

# Check for reasonable data loss
before = df_sales.shape[0]
df_sales = df_sales.dropna(subset=agg_cols)
after = df_sales.shape[0]
print(f'Dropped {before - after} of {before} ({(before - after) / before:.3f}%) records due to missing data.')

# Add in FIPS
df_ia_county_num = pd.read_csv(os.path.join(RDATA, 'iowa_county_numbers.csv'))
df_county_series = df_sales.merge(df_ia_county_num, how='left', left_on='County Number', right_on='county_num')
df_county_series['fips'] = df_county_series['fips'].str.slice(-5).astype(int)

# Agg county-level dataset
agg_map = {
    'Volume Sold (Liters)': np.sum,
    'Invoice/Item Number': len,
    'Bottles Sold': np.sum    
}
rename_map = {
    'Volume Sold (Liters)': 'n_liters',
    'Invoice/Item Number': 'n_transactions',
    'Bottles Sold': 'n_bottles'
}
df_county_series = df_county_series.groupby(['year_month', 'fips'])\
                                   .agg(agg_map)\
                                   .rename(columns=rename_map)

# Feature construction
df_county_series['bottles_p_transaction'] = df_county_series['n_bottles'] / df_county_series['n_transactions']


df_county_series.tail()

Dropped 156796 of 21190664 (0.007%) records due to missing data.


n_liters  n_transactions  n_bottles  bottles_p_transaction
year_month fips                                                             
2021-04    19189   6924.72             954       7605               7.971698
           19191  10051.98            1171      11009               9.401366
           19193  71653.73            7464      96193              12.887594
           19195   2581.05             364       3415               9.381868
           19197   4533.48             557       5526               9.921005

In [7]:
# Next, add in County-level (NOT seasonally adjusted) employment statistics
df_county_emp = pd.read_csv(os.path.join(RDATA, 'county_not_seasonally_adjusted.csv'))
df_county_emp = df_county_emp.loc[df_county_emp['AREATYNAME'] == 'County'].copy()
df_county_emp['AREANAME'] = df_county_emp['AREANAME'].str.replace('County', '').str.strip().str.lower()
df_county_emp['year_month'] = pd.to_datetime(df_county_emp['YEAR'].astype(str) + '-' + df_county_emp['MONTH'])\
                                .apply(lambda x: pd.Period(x, 'M'))

# Add in FIPS
df_ia_county_num = pd.read_csv(os.path.join(RDATA, 'iowa_county_numbers.csv'))
df_ia_county_num['fips'] = df_ia_county_num['fips'].str.slice(-5).astype(int)
df_county_emp = df_county_emp.merge(df_ia_county_num, how='left', left_on='AREANAME', right_on='county')
df_county_emp = df_county_emp.set_index(['year_month', 'fips'])
rename_cols = {
    'EMP': 'employment',
    'UNEMP': 'unemployment',
    'UNEMPRATE': 'unemployment_rate'
}
df_county_emp = df_county_emp.rename(columns=rename_cols)
df_county_emp.tail()

df_county_emp.tail()

,,STFIPS,AREATYNAME,AREANAME,YEAR,MONTH,ADJUSTED,PRELIM,LABORFORCE,employment,unemployment,unemployment_rate,LABFORCE_PARTRATE,county,county_num
year_month,fips,,,,,,,,,,,,,,
2019-05,19191,19,County,winneshiek,2019,May,0,0,12230,11970,260,2.1,NaN,winneshiek,96
2019-06,19191,19,County,winneshiek,2019,June,0,0,11620,11320,300,2.6,NaN,winneshiek,96
2019-07,19191,19,County,winneshiek,2019,July,0,0,11520,11220,290,2.5,NaN,winneshiek,96
2019-08,19191,19,County,winneshiek,2019,August,0,0,11410,11130,280,2.4,NaN,winneshiek,96
2019-09,19191,19,County,winneshiek,2019,September,0,0,12520,12290,230,1.9,NaN,winneshiek,96


In [8]:
# Next, add in County-level covid rates
# Get latest cumulative cases and deaths per month, per county 
# Agg per month at state level

df_cov = pd.read_csv(os.path.join(RDATA, 'us_counties.csv'), parse_dates=['date'])
df_cov['year_month'] = df_cov['date'].dt.to_period('M')
df_cov = df_cov.sort_values('date')
for col in ['cases', 'deaths']:
    df_cov[f'cum_{col}'] = df_cov.groupby(['year_month', 'county', 'state'])[col].transform('last')
df_cov['fips'] = df_cov['fips'] % 100000
df_cov.head()

,date,county,state,fips,cases,deaths,year_month,cum_cases,cum_deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,2020-01,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,2020-01,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,2020-01,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,2020-01,2,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,2020-01,1,0.0


In [9]:
# Get county totals by month
df_county_cov = df_cov.loc[df_cov['state'] == 'Iowa'][['year_month', 'fips', 'cum_cases', 'cum_deaths']]\
                      .drop_duplicates()\
                      .groupby(['year_month', 'fips'], as_index=False)\
                      .agg({'cum_cases': np.sum, 
                            'cum_deaths': np.sum})

# Compute diffs and fillna with initial value
for col in ['cases', 'deaths']:
    df_county_cov[col] = df_county_cov.groupby('fips', as_index=False)[f'cum_{col}'].diff()
    df_county_cov[col] = df_county_cov[col].fillna(df_county_cov[f'cum_{col}'])
  
df_county_cov = df_county_cov.set_index(['year_month', 'fips'])
df_county_cov.tail()

cum_cases  cum_deaths  cases  deaths
year_month fips                                         
2021-05    19189.0       1429        31.0   30.0     0.0
           19191.0       1970        35.0   10.0     2.0
           19193.0      15180       228.0  142.0     1.0
           19195.0        752         8.0   20.0     0.0
           19197.0       1854        37.0   32.0     1.0

In [10]:
# Helper to parse dirty county fips
def fip_parser(x):
    try:
        return int(x)
    except:
        return -1

In [11]:
# Get sociodemographic data

# Population data
pop_cols = {
    'id': 'fips',
    'Estimate!!SEX AND AGE!!Total population': 'pop_total',
    'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
    'Estimate!!SEX AND AGE!!Total population!!18 years and over': 'pop_gt18',
    'Estimate!!SEX AND AGE!!Total population!!21 years and over': 'pop_gt21',
    'Percent!!SEX AND AGE!!Total population!!21 years and over': 'perc_gt21',
    'Percent!!RACE!!Total population!!One race!!White': 'perc_race_white',
    'Percent!!RACE!!Total population!!One race!!Black or African American': 'perc_race_black',
    'Percent!!RACE!!Total population!!One race!!Asian': 'perc_race_asian',
    'Percent!!RACE!!Total population!!Two or more races': 'perc_race_multi'
}

df_acs_pop = pd.read_csv(os.path.join(RDATA, 'ACSDP5Y2019.DP05.csv'), skiprows=1)\
               .rename(columns=pop_cols)
df_acs_pop['fips'] = df_acs_pop['fips'].str.slice(-5).apply(fip_parser)
df_acs_pop = df_acs_pop[list(pop_cols.values())]
df_acs_pop.head()

,fips,pop_total,median_age,pop_gt18,pop_gt21,perc_gt21,perc_race_white,perc_race_black,perc_race_asian,perc_race_multi
0,19001,7085,45.4,5566,5397,76.2,97.5,0.6,0.7,1.0
1,19003,3670,47.3,2903,2813,76.6,98.6,0.1,0.2,0.2
2,19005,13813,44.4,10619,10254,74.2,96.5,1.3,0.6,1.0
3,19007,12452,45.7,9719,9308,74.8,97.0,0.7,0.5,1.5
4,19009,5571,48.1,4425,4262,76.5,96.9,0.2,0.3,2.5


In [12]:
# Load in county lat-long and area data
# Data source: https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html
df_county_geo = pd.read_csv(os.path.join(RDATA, '2020_Gaz_counties_national.txt'), sep='\t')
df_county_geo = df_county_geo.loc[df_county_geo['USPS'] == 'IA']
rename_map = {
    'GEOID': 'fips',
    'ALAND_SQMI': 'area_sq_mi',
    'INTPTLAT': 'lat',
    'INTPTLONG                                                                                                               ': 'long'
}
df_county_geo = df_county_geo.rename(columns=rename_map)[list(rename_map.values())]
df_county_geo.head()

,fips,area_sq_mi,lat,long
790,19001,569.271,41.328528,-94.478164
791,19003,423.433,41.021656,-94.696906
792,19005,639.044,43.274964,-91.382751
793,19007,497.289,40.744249,-92.873065
794,19009,442.961,41.679178,-94.904312


In [13]:
# Economic data
econ_cols = {
    'id': 'fips',
    'Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)': 'median_income',
    'Percent!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Total households!!With Food Stamp/SNAP benefits in the past 12 months': 'perc_snap',
    'Percent!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage': 'perc_health_insurance',
    'Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families': 'perc_poverty'
}

df_acs_econ = pd.read_csv(os.path.join(RDATA, 'ACSDP5Y2019.DP03.csv'), skiprows=1)\
                .rename(columns=econ_cols)
df_acs_econ['fips'] = df_acs_econ['fips'].str.slice(-5).apply(fip_parser)
df_acs_econ = df_acs_econ[list(econ_cols.values())]
df_acs_econ.head()

,fips,median_income,perc_snap,perc_health_insurance,perc_poverty
0,19001,53363,11.7,95.6,5.8
1,19003,49255,10.4,94.2,7.1
2,19005,52216,9.4,92.4,7.4
3,19007,40167,14.4,92.2,11.4
4,19009,52055,8.3,94.8,8.6


In [14]:
# Economic data
soc_cols = {
    'id': 'fips',
    "Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher": 'perc_ed_gt_hs',
    "Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher": 'perc_ed_gt_bach',
    "Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree": 'perc_ed_gt_grad'
}

df_acs_soc = pd.read_csv(os.path.join(RDATA, 'ACSDP5Y2019.DP02.csv'), skiprows=1)\
               .rename(columns=soc_cols)
df_acs_soc['fips'] = df_acs_soc['fips'].str.slice(-5).apply(fip_parser)
df_acs_soc = df_acs_soc[list(soc_cols.values())]
df_acs_soc.head()

,fips,perc_ed_gt_hs,perc_ed_gt_bach,perc_ed_gt_grad
0,19001,94.2,18.5,4.2
1,19003,93.8,15.6,3.6
2,19005,88.3,17.8,4.7
3,19007,90.2,16.3,4.8
4,19009,89.5,17.1,5.1


In [15]:
# Join all features together and store county series dataset
keep_cols = [
    'n_liters',
    'n_transactions',
    'n_bottles',
    'bottles_p_transaction',
    'employment',
    'unemployment',
    'unemployment_rate',
    'cases',
    'deaths'
]

df_county_series_all = df_county_series.join(df_county_emp, how='left')\
                                       .join(df_county_cov, how='left')[keep_cols]\
                                       .reset_index(drop=False)
for df_acs in [df_acs_pop, df_acs_econ, df_acs_soc]:
    df_county_series_all = df_county_series_all.merge(df_acs, how='left', on='fips')
df_county_series_all = df_county_series_all.merge(df_ia_county_num, how='left', left_on='fips', right_on='fips')
df_county_series_all = df_county_series_all.merge(df_county_geo, how='left', on='fips')
df_county_series_all.to_csv(os.path.join(PDATA, 'df_county_series.csv'), index=False)
df_county_series_all.tail()

,year_month,fips,n_liters,n_transactions,n_bottles,bottles_p_transaction,employment,unemployment,unemployment_rate,cases,...,perc_health_insurance,perc_poverty,perc_ed_gt_hs,perc_ed_gt_bach,perc_ed_gt_grad,county,county_num,area_sq_mi,lat,long
11066,2021-04,19189,6924.72,954,7605,7.971698,NaN,NaN,NaN,59.0,...,97.7,6.1,92.4,21.8,5.2,winnebago,95,400.489,43.378124,-93.743488
11067,2021-04,19191,10051.98,1171,11009,9.401366,NaN,NaN,NaN,65.0,...,98.1,4.6,95.2,30.6,11.9,winneshiek,96,689.836,43.292989,-91.850788
11068,2021-04,19193,71653.73,7464,96193,12.887594,NaN,NaN,NaN,494.0,...,94.3,9.7,87.1,23.1,7.2,woodbury,97,872.896,42.393220,-96.053296
11069,2021-04,19195,2581.05,364,3415,9.381868,NaN,NaN,NaN,53.0,...,95.8,6.1,93.3,17.5,4.6,worth,98,400.123,43.373491,-93.248533
11070,2021-04,19197,4533.48,557,5526,9.921005,NaN,NaN,NaN,54.0,...,95.3,7.9,92.4,17.2,4.0,wright,99,580.415,42.733007,-93.734735
